In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from hytana_torch import Hytana
from torchvision.datasets import MNIST
import torch.utils.data.dataloader as dataloader

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.hytana = Hytana()

    def forward(self, x):
        x = self.conv1(x)
        x = self.hytana(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.hytana(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
epochs = 10
log_interval = 10
batch_size = 1024
gamma = 0.7
lr=0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
]), )

# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=batch_size,num_workers=1, pin_memory=True)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [5]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299807
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.014274
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.724051
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.518501
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.446607
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.393042

Test set: Average loss: 0.1619, Accuracy: 9511/10000 (95.1100%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.300653
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.307778
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.262656
Train Epoch: 2 [30720/60000 (51%)]	Loss: 0.242433
Train Epoch: 2 [40960/60000 (68%)]	Loss: 0.259363
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.184626

Test set: Average loss: 0.1068, Accuracy: 9686/10000 (96.8600%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.221984
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.194292
Train Epoch: 3 [20480/60000 (34%)]	Loss: 0.221299
Train Epoch: 3 [30720/60000 (51%)]	Loss: 0.193629
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.181097
Train Epoch: 3 [51200/60000 (85%)